In [1]:
%matplotlib inline

把新闻报道文件放入相应分类名称的文件夹，例如体育类放入sport文件夹。然后用build-sent-datasets.py --sr构建测试数据。最后生成两个文件，train.csv和test.csv。

In [2]:
import torch
import torchtext
import chinese_news_dataset
NGRAMS = 1#汉字每一个token就是一个词
import os
train_dataset, test_dataset = chinese_news_dataset.CN_NEWS(
    root='./data/chinese_news', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

0lines [00:00, ?lines/s]Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/s7/8q3z4z1x61v16fjy9pf98s200000gn/T/jieba.cache
DEBUG:jieba:Dumping model to file cache /var/folders/s7/8q3z4z1x61v16fjy9pf98s200000gn/T/jieba.cache
Loading model cost 0.994 seconds.
DEBUG:jieba:Loading model cost 0.994 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
2066lines [00:12, 165.64lines/s]
2066lines [00:11, 180.27lines/s]
1033lines [00:05, 182.76lines/s]


In [3]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [4]:
VOCAB_SIZE = len(train_dataset.get_vocab())
print(VOCAB_SIZE)
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
print(NUN_CLASS)
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

67872
3


In [5]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1].long() for entry in batch]
#     print(label, text)
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
#     print(label,offsets,text)
    return text, offsets, label

In [6]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [7]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):
    try:
        start_time = time.time()
        train_loss, train_acc = train_func(sub_train_)
        valid_loss, valid_acc = test(sub_valid_)

        secs = int(time.time() - start_time)
        mins = secs / 60
        secs = secs % 60

        print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
        print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
        print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')
    except Exception as ex:
        print('Epoch: %d' %(epoch + 1), ex)

Epoch: 1  | time in 0 minutes, 1 seconds
	Loss: 0.0574(train)	|	Acc: 55.7%(train)
	Loss: 0.0042(valid)	|	Acc: 64.4%(valid)
Epoch: 2  | time in 0 minutes, 1 seconds
	Loss: 0.0173(train)	|	Acc: 91.8%(train)
	Loss: 0.0020(valid)	|	Acc: 85.6%(valid)
Epoch: 3  | time in 0 minutes, 1 seconds
	Loss: 0.0094(train)	|	Acc: 95.7%(train)
	Loss: 0.0081(valid)	|	Acc: 98.1%(valid)
Epoch: 4  | time in 0 minutes, 1 seconds
	Loss: 0.0069(train)	|	Acc: 96.7%(train)
	Loss: 0.0059(valid)	|	Acc: 93.3%(valid)
Epoch: 5  | time in 0 minutes, 1 seconds
	Loss: 0.0053(train)	|	Acc: 98.0%(train)
	Loss: 0.0029(valid)	|	Acc: 95.2%(valid)


In [8]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0000(test)	|	Acc: 96.9%(test)


In [13]:
import re
from torchtext.data.utils import ngrams_iterator
from chinese_news_dataset import get_tokenizer

ag_news_label = {1 : "ent",
                 2 : "sport",
                 3 : "fortune"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("jieba")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = '''
如今，双11晚会早已经落下帷幕，但是，有关晚会的话题量依然是居高不下，热度可见有多高，虽然说昨天的猫晚在最后结束的时候，出现了直播事故，但由于只差几分钟就到零点了，所以，大部分网友都在关注手机，因此，并没有引发太高的争议。
其实，如果除去最后的部分，整台晚会可以说用完美来形容，由于在过去几年打造的猫晚，或多或少都会出现镜头切换等一系列的诸多问题，但今年可以看得出有非常明显的进步。
不仅如此，昨天晚上猫晚的表演嘉宾阵容也是堪称豪华，更是直接碾压隔壁的狮晚，而如今，他们在猫眼上所表演的节目已经被上传至海外的视频网站上，由于国内视频网站的点击量或多或少都会存在一些刷数据的行为，所以，放在海外视频网站，通过点击量才能进一步的验证哪一位的明星在海外的知名度最高，接下来小编就个大家盘点一下排名前五名的嘉宾吧。
'''
vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s " %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a ent 
